# Encoding

In [1]:
cd ..

/home/simonchuth/Desktop/git_repo/patentAI


In [2]:
import tensorflow_hub as hub
from tensorflow_text import SentencepieceTokenizer
import tensorflow as tf

In [3]:
encoder_path = 'encoder/use4_model/'
encoder = hub.load(encoder_path)

In [4]:
import pickle
pkl_path = 'pkl_files/sample_data/ipos_data.pkl'
with open(pkl_path, 'rb') as pkl_file:
    dataset = pickle.load(pkl_file)

In [5]:
import random

In [6]:
random.Random(1).shuffle(dataset)

In [7]:
test_ratio = 0.1
test_size = int(len(dataset) * test_ratio)
test_set = dataset[:test_size]
train_set = dataset[test_size:]
half_train = int(len(train_set)/2)
train_set_1 = train_set[:half_train]
train_set_2 = train_set[half_train:]

In [8]:
from tqdm import tqdm
import re
from tensorflow.keras.preprocessing.text import text_to_word_sequence
import numpy as np
term_pattern = r'".+?"'

In [9]:
end_word = np.zeros((1,512))
end_tensor = tf.convert_to_tensor(end_word)
end_tensor = tf.cast(end_tensor, tf.float32)

In [10]:
def encode(dataset):
    context_tensor = []
    target_tensor = []
    terms = []
    for app in tqdm(dataset):
        intro = app[0]
        claims = ' '.join(app[1])
        context = [intro, claims]
        context = encoder(context)
        context = tf.reshape(context, [1,1024])

        definition = app[2]
        term = re.findall(term_pattern, definition)[0]
        term_tensor = encoder([term])
        definition_tokens = text_to_word_sequence(definition)
        definition_tokens.append('STOPSTOPSTOP')
        for i in range(4, len(definition_tokens)):
            pre_word = ' '.join(definition_tokens[:i])
            target_word = definition_tokens[i]
            pre_word_tensor = encoder([pre_word])
            target_word_tensor = encoder([target_word])
            context_combined = tf.concat([context, term_tensor, pre_word_tensor], axis=1)
            context_tensor.append(context_combined)
            target_tensor.append(target_word_tensor)
            terms.append(term)
    return context_tensor, target_tensor, terms

In [11]:
context_tensor, target_tensor, terms = encode(train_set_2)

100%|██████████| 6066/6066 [16:14<00:00,  6.23it/s]   


In [12]:
train_pkl = 'pkl_files/sample_data/2_train_tensor.pkl'

combine = [context_tensor, target_tensor, terms]
with open(train_pkl, 'wb') as pkl_file:
    pickle.dump(combine, pkl_file)

In [11]:
context_tensor, target_tensor, terms = encode(test_set)
test_pkl = 'pkl_files/sample_data/test_tensor.pkl'

combine = [context_tensor, target_tensor, terms]
with open(test_pkl, 'wb') as pkl_file:
    pickle.dump(combine, pkl_file)

100%|██████████| 1347/1347 [03:37<00:00,  6.20it/s]


# Model building

In [ ]:
from tensorflow.keras import Sequential